In [1]:
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph

n = 1000
tau1 = 2  # Power-law exponent for the degree distribution
tau2 = 1.1  # Power-law exponent for the community size distribution
mu = 0.1  # Mixing parameter
avg_deg = 25  # Average Degree
max_deg = int(0.1 * n)  # Max Degree
min_commu = 60  # Min Community Size
max_commu = int(0.1 * n)  # Max Community Size

G1 = LFR_benchmark_graph(
    n, tau1, tau2, mu, average_degree=avg_deg, max_degree=max_deg, min_community=min_commu, max_community=max_commu,
    seed=7
)

### 去掉 G1 中的重边和自环 
G1 = nx.Graph(G1) # Remove multi-edges

selfloop_edges = list(nx.selfloop_edges(G1)) # a list of self loops

G1.remove_edges_from(selfloop_edges) # Remove self-loops

In [2]:
### LFR 图是有内在的社群结构的，每个节点的社群存储在其 community 属性中，是一个 set
# 通过运行循环，按照内在的社群结构给每个节点一个标签 即为其 intrinsic_membership
# 为了方便 intrinsic_membership 一开始是作为一个 dict 存储的
intrinsic_communities1 = {frozenset(G1.nodes[v]["community"]) for v in G1}
intrinsic_membership1 = {}
for node in range(G1.number_of_nodes()):
    for index, inner_set in enumerate(intrinsic_communities1):
        if node in inner_set:
            intrinsic_membership1[node] = index
            break
# intrinsic_membership = list(intrinsic_membership.values())

# 存储 list 和 clustering 格式的拷贝 省得以后需要再做类型转换了
intrinsic_list1 = list(intrinsic_membership1.values())
from clusim.clustering import Clustering
intrinsic_clustering1 = Clustering(elm2clu_dict={i: [intrinsic_membership1[i]] for i in intrinsic_membership1.keys()})

In [3]:
### 导入 图嵌入评估函数 我自己封装的

from auxpack.evaluate_embd import evaluate_embd as EE

In [4]:
%%time
### NumPy Version
import time
from WGE.lle import lle_np

for D in range(max(intrinsic_list1) + 1 - 2, max(intrinsic_list1) + 1 + 3):
    start_time = time.time()  # Record the start time
    lle_embd = lle_np(G1, D)
    print(f"{D}-dim:", EE(intrinsic_list1, intrinsic_clustering1, lle_embd))
    end_time = time.time()    # Record the end time
    elapsed_time = end_time - start_time
    print(f"Time taken: {elapsed_time:.4f} seconds\n")

10-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
Time taken: 3.0247 seconds

11-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
Time taken: 2.9417 seconds

12-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
Time taken: 2.9349 seconds

13-dim: [1.0, 1.0000000000000002, 1.0, 1.0]
Time taken: 2.7755 seconds

14-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
Time taken: 2.9316 seconds

CPU times: user 5min 27s, sys: 9min 27s, total: 14min 54s
Wall time: 14.6 s


In [5]:
%%time
### Regular Version
from auxpack.lle import lle
for D in range(max(intrinsic_list1)+1-2,max(intrinsic_list1)+1+3):
    lle_embd = lle(G1, D)
    print(f"{D}-dim:", EE(intrinsic_list1, intrinsic_clustering1, lle_embd))

10-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
11-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
12-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
13-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
14-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
CPU times: user 5min 40s, sys: 9min 56s, total: 15min 36s
Wall time: 15 s


In [6]:
%%time
### Sparse Version
from auxpack.lle import lles
for D in range(max(intrinsic_list1)+1-2,max(intrinsic_list1)+1+3):
    lle_embd = lles(G1, D)
    print(f"{D}-dim:", EE(intrinsic_list1, intrinsic_clustering1, lle_embd))

10-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
11-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
12-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
13-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
14-dim: [1.0000000000000002, 1.0, 1.0, 1.0]
CPU times: user 3min 58s, sys: 5min 49s, total: 9min 47s
Wall time: 9.65 s


In [7]:
%%time
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph

n = 10000
tau1 = 2  # Power-law exponent for the degree distribution
tau2 = 1.1  # Power-law exponent for the community size distribution
mu = 0.1  # Mixing parameter
avg_deg = 25  # Average Degree
max_deg = int(0.1 * n)  # Max Degree
min_commu = 60  # Min Community Size
max_commu = int(0.1 * n)  # Max Community Size

G2 = LFR_benchmark_graph(
    n, tau1, tau2, mu, average_degree=avg_deg, max_degree=max_deg, min_community=min_commu, max_community=max_commu,
    seed=7
)

### 去掉 G2 中的重边和自环 
G2 = nx.Graph(G2) # Remove multi-edges

selfloop_edges = list(nx.selfloop_edges(G2)) # a list of self loops

G2.remove_edges_from(selfloop_edges) # Remove self-loops

CPU times: user 1.89 s, sys: 142 ms, total: 2.03 s
Wall time: 1.88 s


In [8]:
### LFR 图是有内在的社群结构的，每个节点的社群存储在其 community 属性中，是一个 set
# 通过运行循环，按照内在的社群结构给每个节点一个标签 即为其 intrinsic_membership
# 为了方便 intrinsic_membership 一开始是作为一个 dict 存储的
intrinsic_communities2 = {frozenset(G2.nodes[v]["community"]) for v in G2}
intrinsic_membership2 = {}
for node in range(G2.number_of_nodes()):
    for index, inner_set in enumerate(intrinsic_communities2):
        if node in inner_set:
            intrinsic_membership2[node] = index
            break
# intrinsic_membership = list(intrinsic_membership.values())

# 存储 list 和 clustering 格式的拷贝 省得以后需要再做类型转换了
intrinsic_list2 = list(intrinsic_membership2.values())
from clusim.clustering import Clustering
intrinsic_clustering2 = Clustering(elm2clu_dict={i: [intrinsic_membership2[i]] for i in intrinsic_membership2.keys()})

In [9]:
%%time
### NumPy Version
import time
from WGE.lle import lle_np

for D in range(max(intrinsic_list2) + 1 - 2, max(intrinsic_list2) + 1 + 3):
    start_time = time.time()  # Record the start time
    lle_embd = lle_np(G2, D)
    print(f"{D}-dim:", EE(intrinsic_list2, intrinsic_clustering2, lle_embd))
    end_time = time.time()    # Record the end time
    elapsed_time = end_time - start_time
    print(f"Time taken: {elapsed_time:.4f} seconds\n")


29-dim: [0.9979999215089495, 0.9943200219084826, 0.9949148532626793, 0.9783675403685586]
Time taken: 307.9072 seconds

30-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
Time taken: 305.5454 seconds

31-dim: [1.0, 1.0000000000000002, 1.0, 1.0]
Time taken: 307.2678 seconds

32-dim: [1.0000000000000002, 1.0, 1.0, 1.0]
Time taken: 302.0074 seconds

33-dim: [1.0, 1.0, 1.0, 1.0]
Time taken: 303.8749 seconds

CPU times: user 13h 54min 45s, sys: 7h 23min 48s, total: 21h 18min 33s
Wall time: 25min 26s


In [10]:
%%time
from WGE.lle import lle_np

D = 16
lle_embd = lle_np(G2, D)
print(f"{D}-dim:", EE(intrinsic_list2, intrinsic_clustering2, lle_embd))

16-dim: [0.9312332698525931, 0.9271461760689117, 0.7685550447628147, 0.8838511491984274]
CPU times: user 2h 47min 59s, sys: 1h 28min 10s, total: 4h 16min 10s
Wall time: 5min 5s


In [11]:
%%time
### Sparse Version
from auxpack.lle import lles
for D in range(max(intrinsic_list2)+1-2,max(intrinsic_list2)+1+3):
    lle_embd = lles(G2, D)
    print(f"{D}-dim:", EE(intrinsic_list2, intrinsic_clustering2, lle_embd))

29-dim: [0.9979999215089493, 0.9944672500943068, 0.9949148532626793, 0.9790325346673042]
30-dim: [1.0000000000000002, 1.0, 1.0, 1.0]
31-dim: [1.0, 1.0000000000000002, 1.0, 1.0]
32-dim: [1.0000000000000002, 1.0, 1.0, 1.0]
33-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
CPU times: user 20min 55s, sys: 1h 25min 34s, total: 1h 46min 29s
Wall time: 1min 50s


In [12]:
%%time
### Regular Version
from auxpack.lle import lle
for D in range(max(intrinsic_list2)+1-2,max(intrinsic_list2)+1+3):
    lle_embd = lle(G2, D)
    print(f"{D}-dim:", EE(intrinsic_list2, intrinsic_clustering2, lle_embd))

29-dim: [0.9979999215089493, 0.9959941792739977, 0.9949148532626793, 0.9853458756023099]
30-dim: [1.0, 1.0, 1.0, 1.0]
31-dim: [1.0000000000000002, 1.0, 1.0, 1.0]
32-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
33-dim: [1.0000000000000002, 1.0000000000000002, 1.0, 1.0]
CPU times: user 14h 27min 35s, sys: 8h 21min 12s, total: 22h 48min 48s
Wall time: 26min 46s


In [1]:
print("天神牛逼")

天神牛逼
